In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q bertopic
!pip install -q bertopic[visualization]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.0 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt-get install g++ openjdk-8-jdk -y
!pip install konlpy
!pip install mecab-python
!apt-get install curl -y
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,688 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,383 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,542 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,099 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:13 https://r2u.stat.i

# 전처리

In [4]:
from datetime import datetime

def parse_datetime(date_string):
   try:
       return datetime.fromisoformat(date_string)
   except ValueError:
       print(f"Invalid date format: {date_string}")
       return None

In [5]:
from konlpy.tag import Mecab
import pandas as pd

mecab = Mecab()

def preprocess(text):
    pos_tagged = mecab.pos(text)

    # 명사 (NNG, NNP 등), 동사, 형용사, 부사 추출
    filtered = [
        word for word, pos in pos_tagged
        if (
            pos.startswith('NN') or  # 명사
            pos.startswith('VV') or  # 동사
            pos.startswith('VA') or  # 형용사
            pos == 'MAG'             # 일반 부사
        ) and len(word) > 1          # 1글자 이상만
    ]

    return filtered



In [7]:
# 데이터프레임 로드
df = pd.read_csv("/content/drive/MyDrive/삼성전자_10000.csv")

df['inp_date'] = df['inp_date'].apply(parse_datetime)

df['content'] = df['content'].fillna('').astype(str)
original_content = df['content'].tolist()

preprocessed_content = [preprocess(preprocessed_content) for preprocessed_content in original_content]
df['preprocessed_content'] = [' '.join(content) for content in preprocessed_content]

In [8]:
len(df)

10000

In [9]:
df.columns.tolist()

['Unnamed: 0',
 'type',
 'inp_date',
 'category',
 'category_sub',
 'origin_nm',
 'origin_url',
 'title',
 'content',
 'extracted_keywords',
 'preprocessed_content']

In [10]:
df[['inp_date', 'preprocessed_content']].head()

,inp_date,preprocessed_content
0,2025-04-03 16:12:00,중국 가전 기업 국내 전자 상거래 전문가 모집 온라인 판매 전략 강화 올해 제품 세...
1,2025-04-03 14:12:00,삼성전자 고등 대학생 학습 도구 활용 태블릿 제품 갤럭시 갤럭시 국내 출시 시리즈 ...
2,2025-04-03 10:54:00,도널드 트럼프 미국 대통령 현지 시간 한국 상호 관세 부과 발표 가운데 반도체 상호...
3,2025-04-03 10:53:00,도널 트럼프 미국 대통령 한국 생산 미국 수입 제품 상호 관세 부과 공식 발표 전자...
4,2025-04-03 10:40:00,도널드 트럼프 미국 대통령 베트남 중국 고율 상호 관세 부과 가운데 해당 국가 공장...


# Mecab과 SBERT를 이용한 Bertopic

In [11]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

import torch
import numpy as np

import random

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

cuda


In [24]:
umap_model = UMAP(
   n_neighbors=15,
   n_components=5,
   min_dist=0.0,
   metric='cosine',
   random_state=SEED
)

hdbscan_model = HDBSCAN(
   min_cluster_size=30, # 1개 주제를 만드려면 최소 3개의 비슷한 문서가 있어야 한다는 말. 적은 문서에서 nr_topics의 지정값이 출력되지 않아서 이를 조정
   metric='euclidean',
   prediction_data=True, # 모델 초기화
   gen_min_span_tree=True,
   cluster_selection_method='eom'
)


embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

vectorizer = CountVectorizer(stop_words=None) # TF-IDF

topic_model = BERTopic(
   language="korean",
   nr_topics=30,
   top_n_words=5,
   calculate_probabilities=True,
   umap_model=umap_model,
   hdbscan_model=hdbscan_model,
   embedding_model=embedding_model,
   vectorizer_model=vectorizer, # TF-IDF to c-TF-IDF
   verbose=True, # 로그 추적
)


content_for_topic = [' '.join(doc) for doc in preprocessed_content]
topics, probs = topic_model.fit_transform(content_for_topic)


# topics는 각 문서가 할당된 주요 토픽의 ID를 담은 배열입니다.
# probs는 각 문서가 모든 토픽에 대해 가지는 확률값을 담은 2D 배열입니다. 크기는 (문서 수 × 토픽 수)입니다.

# calculate_probabilities 옵션의 차이는 다음과 같습니다:
  # True로 설정 시:
  # 각 문서가 각 토픽에 속할 확률을 계산합니다
  # fit_transform()이 토픽 ID와 확률값 두 가지를 반환합니다: topics, probs = fit_transform()
  # 문서가 여러 토픽에 걸쳐 있는 정도를 알 수 있습니다
  # 계산 시간이 더 오래 걸립니다

  # False로 설정 시(기본값):
  # 각 문서를 가장 가능성 높은 하나의 토픽에만 할당합니다
  # fit_transform()이 토픽 ID만 반환합니다: topics = fit_transform()
  # 계산이 더 빠릅니다
  # 다중 토픽 정보는 얻을 수 없습니다

2025-04-09 16:46:16,460 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2025-04-09 16:47:24,031 - BERTopic - Embedding - Completed ✓
2025-04-09 16:47:24,032 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-09 16:47:46,589 - BERTopic - Dimensionality - Completed ✓
2025-04-09 16:47:46,591 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-09 16:47:52,872 - BERTopic - Cluster - Completed ✓
2025-04-09 16:47:52,873 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-04-09 16:47:54,248 - BERTopic - Representation - Completed ✓
2025-04-09 16:47:54,250 - BERTopic - Topic reduction - Reducing number of topics
2025-04-09 16:47:54,270 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-09 16:47:55,599 - BERTopic - Representation - Completed ✓
2025-04-09 16:47:55,603 - BERTopic - Topic reduction - Reduced number of topics from 102 to 30


In [25]:
print(type(topic_model.get_topic_info()))

<class 'pandas.core.frame.DataFrame'>


In [26]:
print(list(topic_model.get_topic_info()))

['Topic', 'Count', 'Name', 'Representation', 'Representative_Docs']


In [ ]:
# index : 토픽 인덱스는 단순히 모델이 내부적으로 할당한 식별자일 뿐, 중요성이나 순위를 나타내지 않습니다.
# Topic: 토픽 ID 번호 (-1은 아웃라이어 토픽)
# Count: 토탈 문서 수 중, 각 토픽에 할당된 문서 수
# Name: 토픽의 자동 생성된 이름 (주요 키워드로 구성)
# Representation: 토픽을 대표하는 주요 단어들 (top_n_words=k에 해당)
# Representative_Docs: 각 토픽을 가장 잘 대표하는 문서 예시, Representative_Docs에서 추출된 정보로 Representation을 만드는 게 아님!!!

# Topic 아웃라이너 : BERTopic에서 토픽 ID가 -1로 표시되는 특별한 분류
# 주요 토픽들에 잘 맞지 않는 문서들의 집합 (일관된 주제가 없거나 여러 주제가 혼합된 문서들이 포함, 아주 독특한 내용을 가진 문서들이 포함)
# BERTopic은 HDBSCAN 클러스터링을 사용하는데, 이 알고리즘은 노이즈가 많거나 데이터 밀도가 낮은 영역의 포인트를 아웃라이어로 분류

In [27]:
print("총 문서 수 : ", topic_model.get_topic_info()['Count'].sum())

총 문서 수 :  10000


In [28]:
topic_df = topic_model.get_topic_info()
topic_df = topic_df[topic_df['Topic'] != -1][['Topic', 'Representation']]
topic_df['Representation'] = topic_df['Representation'].apply(lambda x: ', '.join(x))
topic_df[['Topic', 'Representation']]
topic_df.head()

,Topic,Representation
1,0,"사업, 부회장, 반도체, 삼성전자, 이사"
2,1,"갤럭시, 시리즈, 구독, 스마트폰, 시장"
3,2,"분기, 매출, 메모리, 반도체, 하이닉스"
4,3,"에어컨, 제품, 삼성전자, 기능, 스마트"
5,4,"관세, 미국, 트럼프, 반도체, 생산"


In [29]:
df['topic_id'] = topics
df[['inp_date', 'preprocessed_content', 'topic_id']].head()

,inp_date,preprocessed_content,topic_id
0,2025-04-03 16:12:00,중국 가전 기업 국내 전자 상거래 전문가 모집 온라인 판매 전략 강화 올해 제품 세...,1
1,2025-04-03 14:12:00,삼성전자 고등 대학생 학습 도구 활용 태블릿 제품 갤럭시 갤럭시 국내 출시 시리즈 ...,-1
2,2025-04-03 10:54:00,도널드 트럼프 미국 대통령 현지 시간 한국 상호 관세 부과 발표 가운데 반도체 상호...,4
3,2025-04-03 10:53:00,도널 트럼프 미국 대통령 한국 생산 미국 수입 제품 상호 관세 부과 공식 발표 전자...,4
4,2025-04-03 10:40:00,도널드 트럼프 미국 대통령 베트남 중국 고율 상호 관세 부과 가운데 해당 국가 공장...,4


In [30]:
df['topic_keywords'] = df['topic_id'].map(dict(zip(topic_df['Topic'], topic_df['Representation'])))
# Topic은 모델에서 나온 "토픽 키워드에 대한 ID", Representation은 "토픽 키워드에 대한 ID"에 해당하는 밸류(실제 토픽 키워드)

df[['inp_date', 'preprocessed_content', 'topic_keywords']]

,inp_date,preprocessed_content,topic_keywords
0,2025-04-03 16:12:00,중국 가전 기업 국내 전자 상거래 전문가 모집 온라인 판매 전략 강화 올해 제품 세...,"갤럭시, 시리즈, 구독, 스마트폰, 시장"
1,2025-04-03 14:12:00,삼성전자 고등 대학생 학습 도구 활용 태블릿 제품 갤럭시 갤럭시 국내 출시 시리즈 ...,NaN
2,2025-04-03 10:54:00,도널드 트럼프 미국 대통령 현지 시간 한국 상호 관세 부과 발표 가운데 반도체 상호...,"관세, 미국, 트럼프, 반도체, 생산"
3,2025-04-03 10:53:00,도널 트럼프 미국 대통령 한국 생산 미국 수입 제품 상호 관세 부과 공식 발표 전자...,"관세, 미국, 트럼프, 반도체, 생산"
4,2025-04-03 10:40:00,도널드 트럼프 미국 대통령 베트남 중국 고율 상호 관세 부과 가운데 해당 국가 공장...,"관세, 미국, 트럼프, 반도체, 생산"
...,...,...,...
9995,2025-01-12 13:14:00,올해 미국 라스베이거스 주인공 젠슨 비디아 최고 경영자 키노트 현장 록스타 콘서트 ...,"시크, 기업, 비디아, 모델, 개발"
9996,2025-01-12 13:05:00,가상현실 헤드셋 이제 스마트 안경 주목 경쟁 기술력 정보 기술 기업 관련 제품 시장...,"갤럭시, 시리즈, 구독, 스마트폰, 시장"
9997,2025-01-12 13:05:00,삼성전자 서비스 지난해 최고 서비스 고객 평가 엔지니어 고객 만족 달인 선정 달인 ...,"에어컨, 제품, 삼성전자, 기능, 스마트"
9998,2025-01-12 13:01:00,아모레 퍼시픽 피알 로레알 기술 적용 제품 경쟁력 확보 경쟁 에이피 부스 방문객 기...,"로봇, 기술, 휴머노이드, 통신, 로보"


In [32]:
topic_info = topic_model.get_topic_info()
valid_topics = topic_info[topic_info['Topic'] != -1]
num_topics = valid_topics.shape[0]

print(f"생성된 유효 토픽 수: {num_topics}")

생성된 유효 토픽 수: 29


In [33]:
print("전체 문서 수:", len(topics))
print("토픽별 문서 수 분포:")
print(valid_topics[['Topic', 'Count']])

전체 문서 수: 10000
토픽별 문서 수 분포:
    Topic  Count
1       0   1030
2       1    958
3       2    693
4       3    629
5       4    626
6       5    522
7       6    490
8       7    437
9       8    409
10      9    387
11     10    141
12     11    119
13     12    116
14     13    108
15     14    107
16     15    105
17     16     84
18     17     83
19     18     78
20     19     64
21     20     63
22     21     57
23     22     57
24     23     56
25     24     51
26     25     46
27     26     40
28     27     39
29     28     36


In [34]:
noise_count = topic_info[topic_info['Topic'] == -1]['Count'].values[0]
print(f"잡음(토픽 -1) 문서 수: {noise_count}")
print(f"전체에서 잡음 비율: {noise_count / len(topics):.2%}")

잡음(토픽 -1) 문서 수: 2369
전체에서 잡음 비율: 23.69%


In [38]:
import plotly.graph_objs as go
import random

# 랜덤 색상 생성 함수
def generate_random_colors(n):
    random.seed(42)  # 고정하면 결과 재현 가능
    colors = []
    for _ in range(n):
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        colors.append(f'rgb({r},{g},{b})')
    return colors

# 타임 단위 설정 (기본: 주별, 일별은 아래 주석 해제)
# df['time_unit'] = df['inp_date'].dt.date
df['time_unit'] = df['inp_date'].dt.to_period('W').apply(lambda r: r.start_time)

# 분포 집계
topic_keyword_distribution = df.groupby(['time_unit', 'topic_keywords']).size().unstack(fill_value=0)

# 고유 랜덤 색상 생성
colors = generate_random_colors(len(topic_keyword_distribution.columns))

# 그래프 trace 생성
traces = []
for i, column in enumerate(topic_keyword_distribution.columns):
    trace = go.Scatter(
        x=topic_keyword_distribution.index,
        y=topic_keyword_distribution[column],
        mode='lines+markers',
        name=column,
        line=dict(color=colors[i], width=2),
        marker=dict(size=6)
    )
    traces.append(trace)

# 레이아웃 설정
layout = go.Layout(
    title='토픽 키워드 시계열 분석 (랜덤 색상)',
    xaxis={'title': '날짜'},
    yaxis={'title': '문서 수'},
    hovermode='closest',  # ← 요거만 이렇게 바꿔주세요!
    margin=dict(t=80, b=130),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.45,
        xanchor="center",
        x=0.5,
        font=dict(size=10)
    )
)

fig = go.Figure(data=traces, layout=layout)
fig.show()

# 범례 클릭시 선택적으로 볼 수 있음

# 문서 특징

In [46]:
import pandas as pd
import plotly.express as px

# 🗓️ 날짜만 추출 (시간 제거)
df['date_only'] = df['inp_date'].dt.date

# 📊 날짜별 문서 수 집계
daily_counts = df.groupby('date_only').size().reset_index(name='문서수')

# 📈 선 그래프 시각화
fig = px.line(daily_counts, x='date_only', y='문서수',
              title='📅 날짜별 문서 수',
              labels={'date_only': '날짜', '문서수': '문서 수'},
              markers=True)

fig.update_layout(
    xaxis_title='날짜',
    yaxis_title='문서 수',
    hovermode='x unified'
)

fig.show()

import pandas as pd
import plotly.express as px

# ⏱️ 기준 시작일 (데이터 중 가장 오래된 날짜)
start_date = df['inp_date'].min().normalize()

# ⌛ 며칠 지났는지 계산 → 7일 단위로 그룹 번호 지정
df['custom_week_group'] = ((df['inp_date'] - start_date).dt.days // 7)

# 📅 그룹의 시작 날짜 컬럼 생성 (optional)
df['custom_week_start'] = df['custom_week_group'].apply(lambda x: start_date + pd.Timedelta(days=7 * x))

# 📊 그룹별 문서 수 집계
weekly_custom = df.groupby('custom_week_start').size().reset_index(name='문서수')

# 📈 시각화
fig = px.line(weekly_custom, x='custom_week_start', y='문서수',
              title='🗓️ 사용자 정의 기준 주간 문서 수 (처음 날짜 기준)',
              labels={'custom_week_start': '주 시작일', '문서수': '문서 수'},
              markers=True)

fig.update_layout(
    xaxis_title='주차 (사용자 기준)',
    yaxis_title='문서 수',
    hovermode='x unified'
)

fig.show()

In [47]:
barchart = topic_model.visualize_barchart()
barchart.show()
print('\n')

heatmap = topic_model.visualize_heatmap()
heatmap.show()
print('\n')

hierarchy = topic_model.visualize_hierarchy()
hierarchy.show()
print('\n')

term_rank = topic_model.visualize_term_rank()
term_rank.show()
print('\n')

topics = topic_model.visualize_topics()
topics.show()

### Q) 상승하고 있다고 해도 다른 토픽의 최하점에도 못미치는 경우에는 정말 상승한다고 볼 수 있을것인가?

✅ 방법 1: 상대 상승률 + 절대값 조건 함께 사용
growth_df = growth_df[(growth_df['Recent Avg'] > 10)]  # 예: 최근 평균 문서 수 10 이상
→ 즉, 상승한 건 맞지만 어느 정도 규모는 있어야 진짜 급상승으로 인정

✅ 방법 2: Z-score 기반 이상치 탐지
전체 토픽의 최근 평균 분포를 보고,

평균 대비 얼마나 벗어난 정도인지(표준편차 기준)를 보정하는 방식
from scipy.stats import zscore

growth_df['Z-score'] = zscore(growth_df['Increase'])
growth_df = growth_df[growth_df['Z-score'] > 1.5]

✅ 방법 3: "상대순위 변화" 기반 접근
이전 주차에 비해 현재 전체 토픽 중 순위가 얼마나 올랐는가를 보는 방법

예:

갤럭시가 전체 순위 23위 → 최근 주 5위 → 18단계 상승
→ 이런 방식은 작은 토픽도 고려하되, 무시할 수준의 작음은 제외 가능

In [50]:
#수정필요 실행안해봄
def get_trending_topics(df, n_weeks=3, top_k=5, min_recent_avg=10, min_increase_pct=30, sort_by='increase'):
    """
    최근 n주 동안 급상승한 토픽을 자동으로 리포트해주는 함수

    Parameters:
    - df: 데이터프레임 (주 단위 컬럼 'inp_date', 'topic_keywords' 필요)
    - n_weeks: 최근 비교할 주 수
    - top_k: 리포트할 토픽 수
    - min_recent_avg: 최근 평균 문서 수 최소 기준
    - min_increase_pct: 최소 증가 비율 %
    - sort_by: 정렬 기준 ('increase' or 'increase_pct')

    Returns:
    - 상승 토픽 요약 DataFrame
    """
    import pandas as pd

    df['week'] = df['inp_date'].dt.to_period('W').apply(lambda r: r.start_time)
    weekly_counts = df.groupby(['week', 'topic_keywords']).size().unstack(fill_value=0)

    recent_weeks = weekly_counts.index[-n_weeks:]
    prev_weeks = weekly_counts.index[-2*n_weeks:-n_weeks]

    recent_avg = weekly_counts.loc[recent_weeks].mean()
    prev_avg = weekly_counts.loc[prev_weeks].mean()

    increase = recent_avg - prev_avg
    increase_pct = ((increase) / (prev_avg + 1e-6)) * 100

    growth_df = pd.DataFrame({
        'Prev Avg': prev_avg,
        'Recent Avg': recent_avg,
        'Increase': increase,
        'Increase (%)': increase_pct
    })

    # 조건 필터링
    growth_df = growth_df[
        (growth_df['Recent Avg'] >= min_recent_avg) &
        (growth_df['Increase (%)'] >= min_increase_pct)
    ]

    # 정렬
    if sort_by == 'increase_pct':
        growth_df = growth_df.sort_values(by='Increase (%)', ascending=False)
    else:
        growth_df = growth_df.sort_values(by='Increase', ascending=False)

    return growth_df.head(top_k).round(2)

In [51]:
trending = get_trending_topics(df,
                                n_weeks=3,
                                top_k=5,
                                min_recent_avg=10,
                                min_increase_pct=50,
                                sort_by='increase_pct')

print("🔥 최근 3주간 급상승한 토픽 TOP 5")
print(trending)

🔥 최근 3주간 급상승한 토픽 TOP 5
                        Prev Avg  Recent Avg  Increase  Increase (%)
topic_keywords                                                      
서울, 장례식장, 병원, 빈소, 강남구       0.00       19.00     19.00  1.900000e+09
영통구, 수원시, 컨벤션, 수원, 주주       0.00       13.00     13.00  1.300000e+09
올트먼, 회장, 오픈, 회동, 스타게이트      2.33       52.00     49.67  2.128570e+03
, , , ,                     1.33       13.33     12.00  9.000000e+02
구호, 성금, 산불, 피해, 복구          3.00       12.33      9.33  3.111100e+02


### Q) 동일한 내용의 뉴스가 동시다발적으로 올려진 경우(동일한 뉴스를 뿌린경우)가 존재하는가?

In [54]:
# 'content' 컬럼에서 중복된 값들만 추출 (모든 중복 포함)
duplicated_rows = df[df.duplicated(subset='content', keep=False)]

# 결과 출력
print(duplicated_rows[['content']])


                                                content
54    올 여름 역대급 폭염이 예상되는 가운데 가전업계가 몰려드는 에어컨 수요에 신속 대응...
55    올 여름 역대급 폭염이 예상되는 가운데 가전업계가 몰려드는 에어컨 수요에 신속 대응...
56    올 여름 역대급 폭염이 예상되는 가운데 가전업계가 몰려드는 에어컨 수요에 신속 대응...
57    올 여름 역대급 폭염이 예상되는 가운데 가전업계가 몰려드는 에어컨 수요에 신속 대응...
74    삼성전자가 역대급 폭염 예보와 함께 몰려드는 에어컨 수요에 신속하게 대응하기 위한 ...
...                                                 ...
9929  삼성전자는 자사 2025년형 네오 QLED TV, 라이프스타일 TV 등 주요 모델이...
9937  손태용 삼성전자 영상디스플레이사업부 부사장(왼쪽)과 스벤 오케 VDE 마케팅 영업 ...
9938  손태용 삼성전자 영상디스플레이사업부 부사장(왼쪽)과 스벤 오케 VDE 마케팅 영업 ...
9947  삼성전자 SK하이닉스 등 고대역폭메모리(HBM) 회사가 맞춤형 칩인 '커스텀 HBM...
9972  삼성전자 SK하이닉스 등 고대역폭메모리(HBM) 회사가 맞춤형 칩인 '커스텀 HBM...

[879 rows x 1 columns]


In [57]:
# 중복된 content만 필터링 (모든 중복 포함)
duplicated_rows = df[df.duplicated(subset='content', keep=False)]

# 중복 문서 중에서 고유한 것만 한 번씩만 남기기
unique_duplicates = duplicated_rows.drop_duplicates(subset='content')

# 결과 출력
print(unique_duplicates[['content']])


                                                content
54    올 여름 역대급 폭염이 예상되는 가운데 가전업계가 몰려드는 에어컨 수요에 신속 대응...
74    삼성전자가 역대급 폭염 예보와 함께 몰려드는 에어컨 수요에 신속하게 대응하기 위한 ...
128   삼성전자가 역대급 폭염 예보와 함께 몰려드는 에어컨 수요에 신속하게 대응하기 위해 ...
134   낸드플래시 반도체 가격이 3개월 연속 오르며 메모리 '바닥론'에 힘을 보탰다. 인공...
224                                                    
...                                                 ...
9787  삼성전자가 종합적인 건강 관리 설루션을 제공하는 '갤럭시 링' 신규 사이즈 2종을 ...
9793  대기업들이 바이오를 미래 먹거리로 낙점하고 집중 육성 중인 가운데 삼성과 롯데가 최...
9928  삼성전자는 자사 2025년형 네오 QLED TV, 라이프스타일 TV 등 주요 모델이...
9937  손태용 삼성전자 영상디스플레이사업부 부사장(왼쪽)과 스벤 오케 VDE 마케팅 영업 ...
9947  삼성전자 SK하이닉스 등 고대역폭메모리(HBM) 회사가 맞춤형 칩인 '커스텀 HBM...

[240 rows x 1 columns]


### Q) 데이터 수집 방식의 차이, 기자가 살짝만 수정한 것으로 인해 거의 유사함에도 중복데이터로 처리되지 않은 경우는 없을까?

In [ ]:
## 너무 느려서 faiss로 코드 재구축

from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
from tqdm import tqdm

# 1. 임베딩 모델 준비
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 2. 문서 리스트
docs = df['content'].tolist()

# 3. 임베딩 생성
embeddings = model.encode(docs, convert_to_tensor=True, show_progress_bar=True)

# 4. 유사도 행렬 계산
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)

# 5. 유사도 0.95 이상인 문서 쌍을 기반으로 중복 그룹 만들기
threshold = 0.95
visited = set()
duplicate_groups = []

for i in tqdm(range(len(docs))):
    if i in visited:
        continue
    group = [i]
    for j in range(i + 1, len(docs)):
        if cosine_scores[i][j] >= threshold:
            group.append(j)
            visited.add(j)
    if len(group) > 1:
        duplicate_groups.append(group)
        visited.update(group)

# 6. 중복 그룹 확인
print(f"유사한 문서 그룹 수: {len(duplicate_groups)}")

# 7. 각 그룹에서 하나만 남기고 나머지를 제거 대상으로 기록
to_remove = set()
for group in duplicate_groups:
    # group[0]만 남기고 나머지 제거
    to_remove.update(group[1:])

# 8. 중복된 문서만 따로 추출
semantic_duplicates = df.iloc[list(to_remove)]

# 9. 중복을 제외한 고유한 문서만 추출
df_semantic_dedup = df.drop(index=to_remove).reset_index(drop=True)

# 10. 결과 출력
print(f"중복 문서 수 (유사도 {threshold} 이상): {len(to_remove)}")
print("✅ 고유한 문서 수:", df_semantic_dedup.shape[0])


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

 27%|██▋       | 2713/10000 [08:55<28:32,  4.26it/s]